![Machine Learning Workshop: Content Insights 2020](assets/mlci_banner.jpg)

# Machine Learning Workshop: Content Insights 2020

Welcome to the workshop notebooks!  These notebooks are designed to give you a walk through the steps of creating a model, refining it with user labels, and testing it on content.  You can access the main [workshop forum page](https://INFO_SITE/forums/html/forum?id=241a0b77-7aa6-4fef-9f25-5ea351825725&ps=25), the [workshop files repo](https://INFO_SITE/communities/service/html/communityview?communityUuid=fb400868-b17c-44d8-8b63-b445d26a0be4#fullpageWidgetId=W403a0d6f86de_45aa_8b67_c52cf90fca16&folder=d8138bef-9182-4bdc-8b12-3c88158a219c), or the [symposium home page](https://software.web.DOMAIN) for additional help.

The notebooks are divided into five core components: (A) setup & data, (B) model exploration, (C) labeling, (D) active labeling, (E) and deployment.  You are currently viewing the *setup & data* workbook.

In [1]:
# constants for running the workshop; we'll repeat these in the top line of each workbook.
#   why repeat them? the backup routine only serializes .ipynb files, so others will need 
#   to be downloaded again if your compute instance restarts (a small price to pay, right?)

WORKSHOP_BASE = "https://vmlr-workshop.STORAGE"
# WORKSHOP_BASE = "http://content.research.DOMAIN/projects/mlci_2020"
AGG_METADATA = "agg_metadata.pkl.gz"            # custom file for merged metadata

LQ_JWT = ""  # you need to provide this (copy the string from https://APP_SITE/api/lq/v1/uam/auth)
LQ_ROOT_URL = "https://APP_SITE"
LQ_ROOT_SSL_VERIFY = False

IMDB5000_FEAT = "packages/movie_metadata.csv"   # public dataset for movies

# Notebook C: Self-Service Labeling

Collecting labels can be an arduous, expensive task.  That's why we're turning to an internal platform that simplifies the process with a programmatic API and democrtizes the labelers to some or all of your fellow employees.  In this notebook, we will focus on creating, exploring, and tuning a labeling campaign.  The task herein takes a momentary break away from our contextual advertising focus, but we'll use skill here to continue that direction in the next notebook.

![LabelQuest](assets/labelquest_banner.jpg)

[LabelQuest](https://lq.web.DOMAIN) is an AT&T labeling platform that allows task creation through programmatic API and broad label solicitation across the enterprise.  While due dilligence is still required to avoid senstive information and content, the tracking of labels and compliance-approved usage of the software on desktops, laptops, and tablets is already there.  Additionally, the gamification of the labeling task, manifesting in variance of tasks, a simple but intutitive UX, and the awarding of points and badges may keep spirits up if the number of tasks starts to build up.

# Pythonic API


In [2]:
import lq
from lq.content_label import ContentLabeler

if not LQ_JWT:
    LQ_JWT = ContentLabeler.jwt_load("auth.json")
if not LQ_JWT:
    raise Exception("""
        No token detected (in LQ_JWT), please authenticate and get your JWT token.
        1. Log into the test instance of LQ - https://APP_SITE/
        2a. Get your LQ token from here - https://APP_SITE/api/lq/v1/uam/auth
        2b. OR Save the produced JSON file to the same directory as this script (as auth.json)
    """)

## Project Creation
This cell demonstrates basic functions for listing, retrieving and creating new projects.  To avoid any confusion, the code discussions will use the term `project` instead of `campaign` but otherwise they are meant to be interchangable in this workshop.

The sample below demos these functions...
* `ContentLabeler.list()` - queries active projects with 
* `ContentLabeler.tasks_retrieve()` - query the tasks under a single project with 
* `ContentLabeler.load()` - create or load an existing project
* `ContentLabeler.delete()` - delete a project that has been loaded


In [23]:
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from functools import partial

pd.set_option('display.max_colwidth',1000)

# first, create a handy content labeling instance
labeler = ContentLabeler(LQ_JWT, LQ_ROOT_URL, verbose=False, ssl_verify=LQ_ROOT_SSL_VERIFY)

def proj_create_ex(proj_title="emotion_check"):
    # let's create a simple textual campaign
    labeler.load(proj_title, create_if_missing=True)
    if not labeler.valid:
        raise Exception("""
        Uh oh, failed to create a new project.  Make sure your token is valid and check above errors.""")
    list_classes = ["happy", "sad", "neutral"]
    list_media = ["A juicy apple", "A long-time friend", "In-laws", "Tax day"]
    text_questions = "Please select your primary emotion when thinking of these items"
    num_inserted = labeler.tasks_insert(list_classes, list_media, is_exclusive=True, data_question=text_questions)
    if num_inserted == 0:  # hrm, it says no new inserts, let's check for a prior project
        list_tasks = labeler.tasks_retrieve(0)   # -1=label only, 0=all, 1=unlabeled
        print(f"The project '{proj_title}' already exists, with {len(list_tasks)} tasks of expected {len(list_media)} tasks")
        assert len(list_tasks) == len(list_media)
    else:
        print(f"The project '{proj_title}' was created, with {num_inserted} tasks of expected {len(list_media)} tasks")
        assert num_inserted == len(list_media)
    print(f"That's it for basic project creation, head to LQ to check it out: {LQ_ROOT_URL}")


# create a quick interaction grid for display
def proj_view(create_fn, template_name, proj=None, delete=False, create=False):
    global df_proj
    def refresh():
        df_proj = pd.DataFrame(labeler.list())
        df_proj['id'] = df_proj['title'].map(str) + ": " + df_proj['description'].map(str) 
        df_proj.set_index('id', inplace=True)
        return df_proj
    
    if proj is None:
        df_proj = refresh()
        dropdown.options = list(df_proj.index)
        return
    if not proj in df_proj.index:
        print(f"Error: Couldn't find the specified project {proj} in results!")
        return
    if delete:
        result = labeler.delete()
        df_proj = refresh()
        btn_del.value = False
        dropdown.options = list(df_proj.index)
        dropdown.value = df_proj.index[0]
        return
    if create:
        title_new = f"{template_name}_{len(df_proj)}"
        result = create_fn(title_new)
        df_proj = refresh()
        idx_new = df_proj[df_proj["title"]==title_new].index[0]
        btn_create.value = False
        dropdown.options = list(df_proj.index)
        dropdown.value = idx_new
        return

    # display(f"Matched Tags (for class {class_name}): {df_performance.loc[run_name, 'token']}")
    proj_name = df_proj.loc[proj, "title"]
    labeler.load(proj_name, create_if_missing=False)
    
    if labeler.valid:
        df_tasks = pd.DataFrame(labeler.tasks_retrieve(0))
        df_tasks_sub = df_tasks[['etag', 'data', 'labels']]
        display(df_tasks_sub)
    else:
        print(f"No tasks found for project {proj_name}")

df_proj = None  # start with empty obj

dropdown = widgets.Dropdown(
    options=[] if df_proj is None else list(df_proj.index),  # send run names
    description='Project:', layout={'width': '90%'},
    disabled=False,
)
btn_del = widgets.ToggleButton(description="Delete")
btn_create = widgets.ToggleButton(description="Create")

# create two rows ; "partial" allows us to sub in another function for create
demo_fn = partial(proj_view, proj_create_ex, "new_project")
out = widgets.interactive_output(demo_fn, {'proj':dropdown, 'delete':btn_del, 'create':btn_create})

display(widgets.VBox([dropdown, widgets.HBox([btn_del, btn_create])]), out)
demo_fn()

Output()

## Task Creation
This cell demonstrates how to create a project out of assets that are found in a directory.  To demonstrate, the code will alternate between text-based tasks and image-based ones.

The following methods are demonstrated here.
* (media collection + determination)
* (extra information about the media)

In [21]:

def proj_task_find_ex(proj_title="emotion_check"):
    # let's create a simple textual campaign
    print("IN TASK FIND CREATE")
    labeler.load(proj_title, create_if_missing=True)
    if not labeler.valid:
        raise Exception("""
        Uh oh, failed to create a new project.  Make sure your token is valid and check above errors.""")
    list_classes = ["happy", "sad", "neutral"]
    list_media = ["New task, maybe with HTML"]
    text_questions = "Please select your primary emotion when thinking of these items"
    num_inserted = labeler.tasks_insert(list_classes, list_media, is_exclusive=True, data_question=text_questions)
    if num_inserted == 0:  # hrm, it says no new inserts, let's check for a prior project
        list_tasks = labeler.tasks_retrieve(0)   # -1=label only, 0=all, 1=unlabeled
        print(f"The project '{proj_title}' already exists, with {len(list_tasks)} tasks of expected {len(list_media)} tasks")
        assert len(list_tasks) == len(list_media)
    else:
        print(f"The project '{proj_title}' was created, with {num_inserted} tasks of expected {len(list_media)} tasks")
        assert num_inserted == len(list_media)
    print(f"That's it for basic project creation, head to LQ to check it out: {LQ_ROOT_URL}")


# create two rows ; "partial" allows us to sub in another function for create

demo2_fn = partial(proj_view, proj_task_find_ex, "discovered_tasks")
out = widgets.interactive_output(demo2_fn, {'proj':dropdown, 'delete':btn_del, 'create':btn_create})

display(widgets.VBox([dropdown, widgets.HBox([btn_del, btn_create])]), out)
demo2_fn()


Output()

## Task Type Variance
Demo code to add text, image, or video tasks

# Interface Exploration

## Recommendation by Association
In our user survey, the majority indicated they'd be willing to provide a few labels to a service for better product recommendations, so let's test that promise and its efficacy.  

![willingness to label](assets/labelquest_agreement.jpg)

In the remainder of this notebook, we'll use the [IMDB 5000 dataset](https://www.kaggle.com/carolzhangdc/imdb-5000-movie-dataset) that has a small collection of movies and some metadata about actors and directors.  Unfortunately, you won't be able to make direct requests to WarnerBrothers or HBO to produce your most preferred combination, but this example should demonstrate the power of rapid model refinement with just a few labels.

To jump right to the fun part, we've done a little bit of preprocessing to formulate a few models on the data.  The code in the following cell **will not be executed** because we did it for you, but it's provided here for some fun examples.

1. **Actor Affinity** - finding your preferred actor by the links to others
2. **Genre Preferences** - finding preference for genres by direct categorical links
3. **Crowd Alignment** - how closely do your opinions match that of others, as determined by `likes` or `box office gross`
4. **Embedded Topics** - (advanced) a method that uses our NLP embedding to get recommendations, similar to genre work

In [ ]:
# IMDB5000_FEAT - train some simple models according to above?
df_movies = pd.read_csv(IMDB5000_FEAT)
print(df_movies.columns, len(df_movies))

# task 1: encode all features into movie-based rows
# task 2: select actors by affinity to likes 



df_movies['people'] = df_movies['director_name'].map(str) + "|" + df_movies['actor_1_name'].map(str) + \
                         "|" + df_movies['actor_2_name'].map(str) + "|" + df_movies['actor_3_name'].map(str)
df_movies['people'] = df_movies['people'].apply(lambda x: x.split('|'))
df_movies['genres'] = df_movies['genres'].apply(lambda x: x.split('|'))
df_movies = df_movies.explode('people').explode('genres')

print(df_movies.columns, len(df_movies))
df_movies.head(5)

In [ ]:



# OLD CODE, but examples of thigns that would be run FOR the user to be used in above experiments


import spacy
from spacy.vocab import Vocab
from spacy.tokenizer import Tokenizer
from scipy import spatial
from pathlib import Path

def text2doc(nlp, tag_raw):
    """Given a raw text input, tokenize it to remove stop words"""
    return [x for x in nlp(tag_raw) if not x.is_stop and not x.is_punct]

def doc2vec(nlp, doc, target_domain=None):
    """Given a specific model, clean line of text into an output embedding space"""
    # https://spacy.io/usage/vectors-similarity
    if target_domain is None:
        target_domain = nlp.vocab
    if type(doc) != list:
        doc = [doc]
    tag_doc = None
    for token in doc:
        tag_id = target_domain.strings[token.text]
        if tag_id in target_domain.vectors:   # search existing one
            new_vec = target_domain.vectors[tag_id]
        elif type(token)==str:
            new_vec = nlp(token).vector
        else:
            new_vec = token.vector
        if tag_doc is None:
            tag_doc = new_vec
        else:
            tag_doc += new_vec
    return tag_doc

# doc = text2doc(nlp, "this is a phrase to clean")
# vec = doc2vec(nlp, doc)

# also load our spacy NLP model
nlp = spacy.load('en_core_web_md')
print("NLP model ready to go!")

In [ ]:
# let's plow through our class dataframe and do the cleaning and mapping
# print([x for x in df_classes[df_classes['primary']==1].iloc])
print("Tokenizing and embedding classes...")
list_tokens = [""] * len(df_classes)
list_vect = [np.ndarray((1, nlp.vocab.vectors.shape[1]))] * len(df_classes)
for idx in range(len(df_classes)):  # iterate row indexes
    row = df_classes.iloc[idx]
    # tokenize to remove tags, return tokens
    doc = text2doc(nlp, row["definition"]) + text2doc(nlp, row["class"])
    list_tokens[idx] = [str(x) for x in doc]
    # convert to an embedding array
    list_vect[idx] = doc2vec(nlp, doc)
df_classes["token"] = list_tokens
df_classes["embedding"] = list_vect

# now collect all of the tags that match simple text bag
print("Lookup specific tags by match...")
list_tags = df_flatten['tag'].unique()
map_tokens = {}
embed_tokens = np.ndarray((len(list_tags), nlp.vocab.vectors.shape[1]))
for idx in range(len(list_tags)):   # iterate through full list of known tags
    doc = text2doc(nlp, list_tags[idx])
    for x in doc:  # create map/reference to each term
        x = str(x)
        if x not in map_tokens:  # first time to see this tag?
            map_tokens[x] = []
        map_tokens[x].append(idx)   # save reference to original tag set
    embed_tokens[idx, :] = doc2vec(nlp, doc)  # compute embedding 

# df_classes["token"] = list_tokens
# df_classes["embedding"] = list_vect
print(f"Count of new text-based mapping: {len(map_tokens)}...")
print(f"Shape of embedded tag matrix: {embed_tokens.shape}...")


## A few Labels
What does it look like when someone added labels

## Reranking of Tasks
How do you rerank or prioritize the tasks with label data?

# End of Labeling Material

Ready to label the world?  Just remember that out-of-context, your friends and family may not appreciate you giving them a `happy` or `sad` label, and much less so when you try to click a submit button afterwards! With the familiarity to a labeling system underway, we have the skills to return to task for content labeling.

The next notebook, [notebook D](D_active_labels.ipynb) *(that link may not work)* returns to the contextual ads problem and applies what was learned here to the labeling task that everyone contributed to (hopefully!) before the workshop.
